In [ ]:
# list files (note the "!" to make this a unix/linux command)
# if we only see "sample_data" then this is a new instance and we need to do steps 2 and 3 to get/install files
!ls

In [ ]:
# step 2: set-up spark (NB if Apache amend versions on download site we will need to amend path in wget command)
!clear
!echo welcome
!ls
!rm -f spark-3.3.[01]-bin-hadoop3.tgz* 
!rm -rf spark-3.3.[01]-bin-hadoop3
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
!tar -xf spark-3.3.2-bin-hadoop3.tgz
!ls -alt

In [ ]:
# step 3: wget example file for word count
!wget https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt
!ls

In [ ]:
# init spark (ensure SPARK_HOME set to same version as we download earlier)
!pip3 install findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
# the next line gives us 'local' mode. try 'local[2]' to use 2 cores or 'master:NNNN' to run on Spark standalone cluster at port NNNN
spark_conf = SparkConf().setMaster('local[4]').setAppName('MyApp')
sc = SparkContext(conf=spark_conf)
# see what we have by examining the Spark User Interface
from pyspark.sql import *
from pyspark.sql.functions import *
SparkSession.builder.getOrCreate()

RDD EXAMPLE of wordcount

In [ ]:
text_file = sc.textFile("t8.shakespeare.txt")
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
counts.saveAsTextFile("counts_via_RDD.txt")  # will not overwrite output file

 # in above note the use of "lambda algebra" to operate at low level on RDDs

DF EXAMPLE of wordcount

In [ ]:
spark = SparkSession.builder.appName("words").getOrCreate()


In [ ]:
input = spark.read.text("t8.shakespeare.txt")

In [ ]:
input.show()

In [ ]:
# split input (by line) into words by presuming space is delimiter, set alias "word" to column of results
words = input.select(explode(split(input.value, " ")).alias("word"))

In [ ]:
words.show()

In [ ]:
# group by unique word and count how many
wordCount = words.groupBy("word").count()

In [ ]:
wordCount.show()

In [ ]:
# ... and sort
words.groupBy("word").count().sort("count", ascending=False).show()

In [ ]:
# illustrates orderBy as alternative to sort
words.groupBy("word").count().orderBy("count", ascending=False).show()